In [3]:
import os
import pandas as pd

In [10]:
path = "../test/accuracy"
year = 2014
stacking_classifier = os.listdir(path)
stacking_classifier

['decision_tree', 'logistic_regression', 'random_forest']

In [13]:
file_path = os.path.join(path, stacking_classifier[0], "{0}_2_stacking_accuracy.csv".format(year))
print file_path
pd.read_csv(file_path, header=None, delimiter=";", names=["classifiers", "accuracy"])

../test/accuracy/decision_tree/2014_2_stacking_accuracy.csv


,classifiers,accuracy
0,"(('classification', 'random_forest'), ('regres...",0.613900
1,"(('classification', 'random_forest'), ('regres...",0.616795
2,"(('classification', 'random_forest'), ('classi...",0.613900
3,"(('classification', 'random_forest'), ('classi...",0.614382
4,"(('classification', 'decision_tree'), ('regres...",0.613900
5,"(('classification', 'decision_tree'), ('regres...",0.636583
6,"(('classification', 'decision_tree'), ('regres...",0.610521
7,"(('classification', 'decision_tree'), ('regres...",0.612452
8,"(('classification', 'decision_tree'), ('classi...",0.610521
9,"(('classification', 'decision_tree'), ('classi...",0.610521


In [34]:
dic_accuracy = {}
list_accuracy = []
for n in n_classifiers:
    dic_accuracy[n] = (spark
        .read
        .csv("../test/accuracy/2014_{0}_stacking_accuracy.csv".format(n), sep=";", schema=schema, header=False))
    max_accuracy = dic_accuracy[n].agg(max(col("accuracy")).alias("max")).rdd.map(lambda x: x["max"]).collect()[0]
    print("number classifier: {0}, max accuracy = {1}".format(n, max_accuracy))
    list_accuracy.append((n, max_accuracy))

number classifier: 2, max accuracy = 0.637548262548
number classifier: 3, max accuracy = 0.644787644788
number classifier: 4, max accuracy = 0.655888030888
number classifier: 5, max accuracy = 0.660714285714
number classifier: 6, max accuracy = 0.666988416988
number classifier: 7, max accuracy = 0.670366795367
number classifier: 8, max accuracy = 0.67277992278
number classifier: 9, max accuracy = 0.649613899614


In [40]:
max_classifier = float(np.max(map(lambda x: x[1], list_accuracy)))
n_classifier = filter(lambda x: x[1] == max_classifier, list_accuracy)[0][0]
print("{0}: {1}".format(n_classifier, max_classifier))

8: 0.67277992278


In [46]:
classifiers = (dic_accuracy[n_classifier]
               .filter(col("accuracy") == max_classifier)
               .rdd
               .map(lambda x: x["classifiers"]).collect()[0])


classifiers

u"(('classification', 'logistic_regression'), ('classification', 'random_forest'), ('classification', 'multilayer_perceptron'), ('classification', 'one_vs_rest'), ('regression', 'linear_regression'), ('regression', 'decision_tree'), ('regression', 'random_forest'), ('regression', 'gbt_regressor'))"